In [1]:
import logging

from autogen_agentchat import EVENT_LOGGER_NAME, TRACE_LOGGER_NAME

logging.basicConfig(level=logging.WARNING)

# For trace logging.
trace_logger = logging.getLogger(TRACE_LOGGER_NAME)
trace_logger.addHandler(logging.StreamHandler())
trace_logger.setLevel(logging.DEBUG)

# For structured message logging, such as low-level messages between agents.
event_logger = logging.getLogger(EVENT_LOGGER_NAME)
event_logger.addHandler(logging.StreamHandler())
event_logger.setLevel(logging.DEBUG)

In [2]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
# model_client = OpenAIChatCompletionClient(
#     model="gpt-4o-2024-08-06",
#     # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
# )

from autogen_core.models import ModelFamily
model_client = OpenAIChatCompletionClient(
    model="qwen2.5:7b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": ModelFamily.ANY,
    },
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

await Console(team.run_stream(task="Write a short poem about the fall season."))  # Stream the messages to the console.

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Leaves of amber and russet, falling,
Whisper to ground in autumn's calling.
Chill winds dance, a crisp inviting caress,
As twilight's embrace with dusk it presses.

Sunlight slants, through bare branches,
Painting glows on paths that once held masses.
Hues of orange and red, they tell the tale,
Of harvest's bounty, soon to pale.

Pumpkins grin under skies of endless blue,
Cider warms hands as cool breezes brew.
Nature's symphony in a fleeting score,
Fall's magic in twilight and morning's shore.
---------- critic ----------
Your poem captures the essence of fall beautifully with vivid imagery and heartfelt emotion. The transitions between the lines flow well, and each stanza focuses on different aspects of autumn—leaves, sunlight, warmth, and nature's beauty. To enhance it further, perhaps consider adding more auditory elements like rustling leaves or falling into stanzas to create a stron

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=121), metadata={}, content="Leaves of amber and russet, falling,\nWhisper to ground in autumn's calling.\nChill winds dance, a crisp inviting caress,\nAs twilight's embrace with dusk it presses.\n\nSunlight slants, through bare branches,\nPainting glows on paths that once held masses.\nHues of orange and red, they tell the tale,\nOf harvest's bounty, soon to pale.\n\nPumpkins grin under skies of endless blue,\nCider warms hands as cool breezes brew.\nNature's symphony in a fleeting score,\nFall's magic in twilight and morning's shore.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=76), metadata={}, content="Your poem captures the essence of fall beautifully with vivid imagery a